# Test carta

## Libreria $\texttt{ipyleaflet}$

### Import librerie

In [1]:
%pip install ipyleaflet
%pip install ipywidgets
%pip install jupyterlab_widgets

# %jupyter nbextension enable --py widgetsnbextension
# %jupyter nbextension enable --py ipyleaflet

Defaulting to user installation because normal site-packages is not writeable


    keyring (<=21.8.*,>=19)
             ~~~~~~~^


    urllib3 (<1.26,>=1.25.4numpy>=1.14.0) ; extra == 'tests'
            ~~~~~~~~~~~~~~~^


Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable


...

Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable


    keyring (<=21.8.*,>=19)
             ~~~~~~~^


    urllib3 (<1.26,>=1.25.4numpy>=1.14.0) ; extra == 'tests'
            ~~~~~~~~~~~~~~~^


Note: you may need to restart the kernel to use updated packages.


### Carta semplice

In [2]:
# from ipyleaflet import Map
import ipyleaflet as ll
from ipywidgets import Button, VBox

from IPython.display import FileLink

#> ------------------------------------------------------------------
#> Map
m = ll.Map(center=(45.0, 10.0), zoom=5)  # , basemap=...

#> ------------------------------------------------------------------
#> Layers

#> ------------------------------------------------------------------
#> Controls
#> Scale
m.add(ll.ScaleControl(position='bottomleft'))
#> Full screen
m.add(ll.FullScreenControl())
#> Measure
m.add(ll.MeasureControl(position='bottomleft', active_color='orange', primary_length_unit='kilometers'))
#> Split map, with a slider
#> Draw, GeomanDrawControl
draw_control = ll.GeomanDrawControl()
#> Legend
#> Search



# Store drawn data
drawn_features = []

# Capture new feature additions
def handle_draw(action, geo_json):
    if action == 'created':
        drawn_features.append(geo_json)
        print(f"Added new feature: {geo_json['geometry']}")

draw_control.on_draw(handle_draw)
m.add_control(draw_control)

# Add Download Button
download_button = Button(description="Download Features")

def download_geojson(_):
    # Save the drawn features to a GeoJSON file
    geojson_data = {
        "type": "FeatureCollection",
        "features": drawn_features
    }
    file_path = "drawn_features.geojson"
    with open(file_path, "w") as f:
        json.dump(geojson_data, f)
        
    print(f"Features saved to {file_path}")
    display(FileLink(file_path))

download_button.on_click(download_geojson)
# download_button.on_click(lambda _: display(FileLink(download_geojson)))

# Display Map and Button
VBox([m, download_button])


# m

In [3]:
import os
print(f"cwd: {os.getcwd()}")

cwd: /home/davide/Documents/basics-books/books/bbooks-insights/ch


### Carta con qualche dettaglio e interattività

In [4]:

from ipyleaflet import (
    Map, GeoJSON, LayerGroup, WidgetControl
)
from ipywidgets import IntSlider, VBox, HTML
import json


# Load GeoJSON data (replace with real historical boundary data)
# Here we use dummy data, you need to replace these with actual files for Italy and France boundaries over time
def load_geojson(country, year):
    # This function simulates loading GeoJSON files
    # Replace it with paths to actual files or GeoJSON objects
    return {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [2.0, 47.0], [7.0, 47.0], [7.0, 42.0], [2.0, 42.0], [2.0, 47.0]
                        ] if country == "France" else
                        [[8.0, 46.0], [14.0, 46.0], [14.0, 40.0], [8.0, 40.0], [8.0, 46.0]]
                    ],
                },
                "properties": {
                    "country": country,
                    "year": year
                }
            }
        ]
    }

# Map Initialization
m = Map(center=(45, 10), zoom=5)

# Slider for Year
year_slider = IntSlider(value=1800, min=1800, max=1950, step=10, description='Year:')

# Layer Groups for Countries
italy_layer_group = LayerGroup(name="Italy")
france_layer_group = LayerGroup(name="France")

# Function to Update Layers Based on Year
def update_layers(change):
    year = change['new']
    italy_layer_group.clear_layers()
    france_layer_group.clear_layers()

    italy_data = load_geojson("Italy", year)
    france_data = load_geojson("France", year)

    italy_geojson = GeoJSON(data=italy_data, style={'color': 'green', 'opacity': 0.6, 'weight': 1.5})
    france_geojson = GeoJSON(data=france_data, style={'color': 'blue', 'opacity': 0.6, 'weight': 1.5})

    italy_layer_group.add_layer(italy_geojson)
    france_layer_group.add_layer(france_geojson)

# Initial Layer Update
update_layers({'new': year_slider.value})

# Attach Slider Event
year_slider.observe(update_layers, 'value')

# Add Layers to the Map
m.add_layer(italy_layer_group)
m.add_layer(france_layer_group)

# Add Controls to the Map
m.add_control(WidgetControl(widget=year_slider, position='topright'))

# Display Map
m


Map(center=[45, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…